# Prepare from bootstrap paths

Now we will use the initial trajectories we obtained from bootstrapping to run an MSTIS simulation. This will show both how objects can be regenerated from storage and how regenerated equivalent objects can be used in place of objects that weren't stored.

Tasks covered in this notebook:
* Loading OPS objects from storage
* Ways of assigning initial trajectories to initial samples
* Setting up a path sampling simulation with various move schemes
* Visualizing trajectories while the path sampling is running

In [1]:
%matplotlib inline
import openpathsampling as paths
import numpy as np
import math

In [2]:
# Imports for plotting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from matplotlib.legend_handler import HandlerLine2D

### Loading things from storage

First we'll reload some of the stuff we stored before. Of course, this starts with opening the file.

In [3]:
old_store = paths.AnalysisStorage("ala.nc")

A lot of information can be recovered from the old storage, and so we don't have the recreate it. However, we did not save our network, so we'll have to create a new one. Since the network creates the ensembles, that means we will have to translate the trajectories from the old ensembles to new ensembles.

In [4]:
print "PathMovers:", len(old_store.pathmovers)
print "Engines:", len(old_store.engines)
print "Samples:", len(old_store.samples)
print "Ensembles:", len(old_store.ensembles)
print "SampleSets:", len(old_store.samplesets)
print "Snapshots:", len(old_store.snapshots)
print "Networks:", len(old_store.networks)

PathMovers: 0
Engines: 3
Samples: 28
Ensembles: 351
SampleSets: 1
Snapshots: 5982
Networks: 1


Loading from storage is very easy. Each store is a list. We take the 0th snapshot as a template (it doesn't actually matter which one) for the next storage we'll create. There's only one engine stored, so we take the only one.

In [5]:
template = old_store.snapshots[0]

In [6]:
engine = old_store.engines['default']

In [7]:
mstis = old_store.networks[0]

Now we need to set up real trajectories that we can use for each of these. We can start by loading the stored sample set.

In [8]:
sset = paths.SampleSet(old_store.samplesets[0])

In [9]:
sset.sanity_check()
assert(len(sset)==28)

### Setting up special ensembles

Whichever way we initially set up the `SampleSet`, at this point it only contains samples for the main sampling trajectories of each transition. Now we need to put trajectories into various auxiliary ensembles.

#### Multiple state outer ensemble

The multiple state outer ensemble is, in fact, sampled during the bootstrapping. However, it is actually sampled once for every state that shares it. It is very easy to find a trajectory that satisfies the ensemble and to load add that sample to our sample set.

In [10]:
for outer_ens in mstis.special_ensembles['ms_outer']:
    # doesn't matter which we take, so we take the first
    traj = next(s.trajectory for s in old_store.samplesets[0] if outer_ens(s.trajectory)==True)
    samp = paths.Sample(
            replica=None,
            ensemble=outer_ens,
            trajectory=traj
    )
    # now we apply it and correct for the replica ID
    sset.append_as_new_replica(samp)

In [11]:
sset.sanity_check()
assert(len(sset)==29)

#### Minus interface ensemble

The minus interface ensembles do not yet have a trajectory. We will generate them by starting with same-state trajectories (A-to-A, B-to-B, C-to-C) in each interface, and extending into the minus ensemble.

* check whether the traj is A-to-A
* extend

First we need to make sure that the trajectory in the innermost ensemble of each state also ends in that state. This is necessary so that when we extend the trajectory, it can extends into the minus ensemble.

If the trajectory isn't right, we run a shooting move on it until it is.

In [12]:
for transition in mstis.sampling_transitions:
    innermost_ensemble = transition.ensembles[0]
    shooter = None
    if not transition.stateA(sset[innermost_ensemble].trajectory[-1]):
        shooter = paths.OneWayShootingMover(ensemble=innermost_ensemble,
                                            selector=paths.UniformSelector(),
                                            engine=engine)
        pseudoscheme = paths.LockedMoveScheme(root_mover=shooter)
        pseudosim = paths.PathSampling(storage=None, 
                                       move_scheme=pseudoscheme, 
                                       globalstate=sset)
    while not transition.stateA(sset[innermost_ensemble].trajectory[-1]):
        pseudosim.run(1)
        sset = pseudosim.globalstate

    

Now that all the innermost ensembles are safe to use for extending into a minus interface, we extend them into a minus interface:

In [13]:
def traj_info(traj, ensemble):
    return traj.summarize_by_volumes_str(
            {"A" : ensemble.state_vol,
             "I" : ~ensemble.state_vol & ensemble.innermost_vol,
             "X" : ~ensemble.innermost_vol})

In [14]:
minus_samples = []
for transition in mstis.sampling_transitions:
#    paths.tools.refresh_output('Creating minus sample for %s' % transition.name) 
    print transition.name,
    choices = list()
    from_state = transition.stateA
    max_length = 10
    while len(choices) == 0: 
        leaveSt = paths.SequentialEnsemble([
                paths.AllInXEnsemble(from_state),
                paths.SingleFrameEnsemble(
                    paths.AllOutXEnsemble(from_state)
                )
            ])
        extend = paths.LengthEnsemble(slice(0,max_length)) & paths.SequentialEnsemble([
                paths.AllOutXEnsemble(from_state),
                paths.SingleFrameEnsemble(
                    paths.AllInXEnsemble(from_state)
                )
            ])
        part1 = sset[transition.ensembles[0]].trajectory
        part2 = engine.generate_forward(part1[-1], leaveSt)
        part3 = engine.generate_forward(part2[-1], extend)
        
#        print len(part1), len(part2), len(part3)
        
        attempt = part1 + part2[1:] + part3[1:]
        
#         attempt = transition.minus_ensemble.populate_minus_ensemble(
#             partial_traj=sset[transition.ensembles[0]].trajectory,
#             minus_replica_id=-len(minus_samples)-1,
#             engine=engine
#         )
        choices = transition.minus_ensemble.split(attempt)
    
        print '%2d' % max_length,
        
        max_length += 5

    print '(%2d) %s' % (len(attempt), traj_info(attempt, transition.minus_ensemble))
        
    minus_samples.append(
        paths.Sample(
            replica=-len(minus_samples)-1,
            trajectory=choices[0],
            ensemble=transition.minus_ensemble
        )        
    )
    
sset = sset.apply_samples(minus_samples)

Out C0 10 15 20 25 (89) A-X-A-X-A
Out B0 10 15 20 25 30 35 40 45 50 55 60 65 70 75 (66) A-X-A-X-A
Out A0 10 (14) A-X-A-X-A
Out D0 10 (22) A-X-A-X-A
Out E0 10 15 (24) A-X-A-X-A
Out F0 10 (17) A-X-A-X-A


In [15]:
sset.sanity_check()
assert(len(sset)==35)

In [16]:
for s in sset:
    print s.replica, len(s.trajectory), s.ensemble.name, s.ensemble(s.trajectory)

0 7 I'face 0 True
1 48 I'face 1 True
2 90 I'face 2 True
3 84 I'face 3 True
4 84 [TISEnsemble] True
5 67 I'face 0 True
6 52 I'face 1 True
7 87 I'face 2 True
8 87 [TISEnsemble] True
9 6 I'face 0 True
10 66 I'face 1 True
11 92 I'face 2 True
12 48 I'face 3 True
13 48 [TISEnsemble] True
14 11 I'face 0 True
15 11 I'face 1 True
16 48 I'face 2 True
17 26 I'face 3 True
18 26 [TISEnsemble] True
19 16 I'face 0 True
20 17 I'face 1 True
21 15 I'face 2 True
22 15 [TISEnsemble] True
23 15 I'face 0 True
24 15 I'face 1 True
25 33 I'face 2 True
26 71 I'face 3 True
27 71 [TISEnsemble] True
28 90 [UnionEnsemble] True
-1 89 [MinusInterfaceEnsemble] True
-2 66 [MinusInterfaceEnsemble] True
-3 14 [MinusInterfaceEnsemble] True
-4 22 [MinusInterfaceEnsemble] True
-5 24 [MinusInterfaceEnsemble] True
-6 17 [MinusInterfaceEnsemble] True


## Equilibration

In molecular dynamics, you need to equilibrate if you don't start with an equilibrium frame (e.g., if you start with solvent molecules on a grid, your system should equilibrate before you start taking statistics). Similarly, if you start with a set of paths which are far from the path ensemble equilibrium, you need to equilibrate. This could either be because your trajectories are not from the real dynamics (generated with metadynamics, high temperature, etc.) or because your trajectories are not representative of the path ensemble (e.g., if you put transition trajectories into all interfaces).

As with MD, running equilibration can be the same process as running the total simulation. However, in path sampling, it doesn't have to be: we can equilibrate without replica exchange moves or path reversal moves, for example. In the example below, we create a `MoveScheme` that only includes shooting movers.

In [17]:
equil_scheme = paths.MoveScheme(mstis)
import openpathsampling.analysis.move_strategy as strat
equil_scheme.append([
        strat.OneWayShootingStrategy(engine=engine), 
        strat.OrganizeByMoveGroupStrategy()
    ])

In [18]:
equilibration = paths.PathSampling(
    storage=None,
    globalstate=sset,
    move_scheme=equil_scheme
)

In [19]:
equilibration.run(20)

Working on Monte Carlo cycle number 20.
DONE! Completed 20 Monte Carlo cycles.


In [20]:
sset = equilibration.globalstate

In [22]:
import openpathsampling as paths

In [23]:
storage = paths.storage.Storage("weina_pre.nc", "w")

In [24]:
storage.save(template)

(store.snapshots[BaseSnapshot],
 2,
 UUID('52e49e51-4765-11e6-a326-000000000042'))

In [25]:
storage.snapshots.variables['uuid'][:]

array([u'52e49e51-4765-11e6-a326-000000000042'], dtype=object)

In [7]:
tt = old_store.trajectories[0]

In [8]:
for snap in tt.iter_proxies():
    i = snap._idx
    
    print snap._store is old_store.snapshots, i, old_store.snapshots.pos(i), storage.snapshots.pos(i)

True 52e49e51-4765-11e6-a326-00000000000c 4 None
True 52e49e51-4765-11e6-a326-00000000013a 6 None
True 52e49e51-4765-11e6-a326-000000000154 8 None
True 52e49e51-4765-11e6-a326-00000000016e 10 None
True 52e49e51-4765-11e6-a326-000000000188 12 None
True 52e49e51-4765-11e6-a326-0000000001a2 14 None
True 52e49e51-4765-11e6-a326-0000000001bc 16 None
True 52e49e51-4765-11e6-a326-0000000001d6 18 None


In [9]:
storage.trajectories.save(tt)

UUID('52e49e51-4765-11e6-a326-000000000120')

In [26]:
storage.save(sset)
storage.save(mstis)
storage.save(engine)

(store.engines[DynamicsEngine],
 11,
 UUID('52e49e51-4765-11e6-a326-000000000010'))

In [27]:
for tag in ['states', 'state_centers', 'state_letter', 'interface_levels']:
    storage.tag[tag] = old_store.tag[tag]

In [28]:
storage.tag['engine'] = engine
storage.tag['initial_sampleset'] = sset
storage.tag['mstis'] = mstis

In [29]:
import os

In [30]:
print 'filesize : %d MB' % (os.stat('weina_pre.nc').st_size / 1024 / 1024)

filesize : 37 MB


In [31]:
storage.close()